In [60]:
import open3d as o3d
import numpy as np
import os
from tqdm import tqdm
from time import sleep

## Fix your path here

In [61]:
filepath_kri = ["/Users/kristofferkjaer/Desktop/SPRING_MALE", "/Users/kristofferkjaer/Desktop/mesh"]

In [62]:
data_paths = filepath_kri

files = []
pcds_females = []
pcds_males = []
pcds = [pcds_females, pcds_males]

for data in range(len(data_paths)):
    files = []
    folder_to_view = data_paths[data]
    list = pcds[data]
    for file in os.listdir(folder_to_view):
        files.append(file)
    for i in tqdm(range(int((5)))):
        sleep(0.0000000000001)
        file = (folder_to_view + "/" + str(files[i]))
        mesh = o3d.io.read_triangle_mesh(file)
        pcd = mesh.sample_points_poisson_disk(2048)
        list.append(pcd)

100%|██████████| 5/5 [00:02<00:00,  2.03it/s]


### Converts mesh to point cloud and visualise

In [63]:
def normalize(points): # The input of this function is a point cloud object
    # This function normalizes a point cloud within a unit sphere
    points = np.asarray(points.points) 
    centroid = np.mean(points, axis=0)
    points -= centroid
    furthest_distance = np.max(np.sqrt(np.sum(abs(points)**2,axis=-1)))
    points /= furthest_distance
    return points # The function return a numpy array of dimension nx3 where the points are normalized within a unit sphere
